In [11]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.llm
import clin.eval.med_status
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
# results_dir = '../results_tmp/'
from clin.config import PATH_REPO
import datasets
import imodelsx.viz
import imodelsx.process_results
def viz_blue_rows(df):
    return (
        df
        # highlight rows 2 and 4 including the index
        .style.apply(lambda x: ['background: lightblue' if 'self-verified' in x.name else '' for i in x], axis=1)
        # bold index label for row 2
        .apply(lambda x: ['font-weight: bold' if 'self-verified' in x.name else '' for i in x], axis=1)
        .format(precision=3)
    )   
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[r.checkpoint != 'chaoyi-wu/PMC_LLAMA_7B']
r = r[r.n_shots == 5]
r = r[[col for col in r.columns if not col in ['checkpoint_verify', 'role_verify']]]
# r = imodelsx.process_results.average_over_seeds(r, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 60/60 [00:00<00:00, 1867.62it/s]


In [12]:
r_med = r[r.dataset_name == 'medication_status']
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
# dfe = df.iloc[nums]
r_med = clin.eval.med_status.add_status_eval(r_med, df)

cols = {
    'f1___original': 'Medication name, original',
    'f1___ov_pv_ev': 'Medication name, self-verified',
    'status_f1_macro_cond___original': 'Medication status, original',
    'status_f1_macro_cond___sv': 'Medication status, self-verified',
}
rt_med_status = r_med.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_med_status_sem = r_med.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_med_status)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

checkpoint,gpt-3.5-turbo,gpt-4-0314,text-davinci-003
"Medication name, original",0.892,0.884,0.929
"Medication name, self-verified",0.898,0.910,0.935
"Medication status, original",0.194,0.139,0.561
"Medication status, self-verified",0.346,0.456,0.581


In [13]:
r_ebm = r[r.dataset_name == 'ebm']
cols = {
    'f1___original': 'Clinical trial arm, original',
    'f1___ov_pv_ev': 'Clinical trial arm, self-verified',
}
rt_ebm = r_ebm.groupby(['checkpoint'])[list(cols.keys())].mean().rename(columns=cols).round(3).T
rt_ebm_sem = r_ebm.groupby(['checkpoint'])[list(cols.keys())].sem(ddof=0).rename(columns=cols).round(3).T
viz_blue_rows(rt_ebm)

checkpoint,gpt-3.5-turbo,gpt-4-0314,text-davinci-003
"Clinical trial arm, original",0.342,0.419,0.512
"Clinical trial arm, self-verified",0.456,0.530,0.575


In [14]:
for k in rt_ebm.columns:
    rt_ebm[k] = rt_ebm[k].astype(str) + "\err{" + rt_ebm_sem[k].astype(str) + "}"
for k in rt_med_status.columns:
    rt_med_status[k] = (
        rt_med_status[k].astype(str) + "\err{" + rt_med_status_sem[k].astype(str) + "}"
    )
rt = pd.concat((rt_ebm, rt_med_status))
rt.columns = rt.columns.map(lambda x: imodelsx.viz.CHECKPOINTS_RENAME_DICT.get(x, x))
viz_blue_rows(rt)

<>:2: DeprecationWarning: invalid escape sequence '\e'
<>:5: DeprecationWarning: invalid escape sequence '\e'
<>:2: DeprecationWarning: invalid escape sequence '\e'
<>:5: DeprecationWarning: invalid escape sequence '\e'
/tmp/ipykernel_1051835/2639324598.py:2: DeprecationWarning: invalid escape sequence '\e'
  rt_ebm[k] = rt_ebm[k].astype(str) + "\err{" + rt_ebm_sem[k].astype(str) + "}"
/tmp/ipykernel_1051835/2639324598.py:5: DeprecationWarning: invalid escape sequence '\e'
  rt_med_status[k].astype(str) + "\err{" + rt_med_status_sem[k].astype(str) + "}"


checkpoint,ChatGPT,GPT-4,GPT-3
"Clinical trial arm, original",0.342\err{0.01},0.419\err{0.008},0.512\err{0.009}
"Clinical trial arm, self-verified",0.456\err{0.007},0.53\err{0.01},0.575\err{0.003}
"Medication name, original",0.892\err{0.004},0.884\err{0.003},0.929\err{0.002}
"Medication name, self-verified",0.898\err{0.002},0.91\err{0.001},0.935\err{0.001}
"Medication status, original",0.194\err{0.01},0.139\err{0.011},0.561\err{0.058}
"Medication status, self-verified",0.346\err{0.025},0.456\err{0.048},0.581\err{0.062}


In [15]:
print(rt.style.format(precision=3).to_latex(hrules=True))

\begin{tabular}{llll}
\toprule
checkpoint & ChatGPT & GPT-4 & GPT-3 \\
\midrule
Clinical trial arm, original & 0.342\err{0.01} & 0.419\err{0.008} & 0.512\err{0.009} \\
Clinical trial arm, self-verified & 0.456\err{0.007} & 0.53\err{0.01} & 0.575\err{0.003} \\
Medication name, original & 0.892\err{0.004} & 0.884\err{0.003} & 0.929\err{0.002} \\
Medication name, self-verified & 0.898\err{0.002} & 0.91\err{0.001} & 0.935\err{0.001} \\
Medication status, original & 0.194\err{0.01} & 0.139\err{0.011} & 0.561\err{0.058} \\
Medication status, self-verified & 0.346\err{0.025} & 0.456\err{0.048} & 0.581\err{0.062} \\
\bottomrule
\end{tabular}

